# Foundations of Computer Science 

Developers: 


Alessandro Motta 812309

Mattia Ventola 812475

In [12]:
import pandas as pd
import re
import datetime
import numpy as np
from pandas.tseries import offsets

In [13]:
loans_len=pd.read_csv('loans_lenders.csv')
loans=pd.read_csv('loans.csv')
lenders=pd.read_csv('lenders.csv')
c_stat=pd.read_csv('country_stats.csv')

# Punto 1:

Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.


In [14]:
loans_len.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [15]:
#Selezioniamo solamente l'Id del loan
loan_id=loans_len['loan_id']
loan_id

0           483693
1           483738
2           485000
3           486087
4           534428
            ...   
1387427     678999
1387428    1207353
1387429    1206220
1387430    1206425
1387431    1206486
Name: loan_id, Length: 1387432, dtype: int64

In [16]:
# con la funzione explode abbiamo separato le stringhe dei lenders ogni volta è presente la virgola attribuendole allo stesso indice
sing_lend=loans_len['lenders'].str.split(',').explode()
sing_lend

0                      muc888
0                     sam4326
0                 camaran3922
0                 lachheb1865
0                 rebecca3499
                  ...        
1387430     trogdorfamily7622
1387430             danny6470
1387430               don6118
1387431              alan5175
1387431           amy38101311
Name: lenders, Length: 28293931, dtype: object

In [17]:
# la funzione strip è servita per eliminare gli spazi bianchi tra i lender
sing_lend=sing_lend.str.strip()

In [18]:
# abbiamo creato il dataset assegnando ad ogni lender il suo loan iniziale
loan_id=pd.Series(loans_len['loan_id'])

ab={ 'loan_id': loan_id , 'lenders': sing_lend}
fin_fin=pd.DataFrame(ab)

In [19]:
fin_fin

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499
...,...,...
1387430,1206425,trogdorfamily7622
1387430,1206425,danny6470
1387430,1206425,don6118
1387431,1206486,alan5175


In [20]:
fin_fin = fin_fin.reset_index(drop=True)
fin_fin

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


# Punto 2: 


For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.


In [21]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [22]:
# notiamo che molte date presentano un disburse time maggiore dell'expiration time 
# crediamo che questo sia un errore in quanto la data scadenza del prestito è precedente a quella di esborso
loans.loc[loans['disburse_time'] > loans['planned_expiration_time']]

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
63,1158981,Rabiu,English,Rabiu is a Babban Gona maize farmer from Kwany...,Rabiu is a Babban Gona maize farmer from Kwany...,250.0,250.0,funded,Personal Expenses,Personal Use,...,2016-10-31 05:10:53.000 +0000,15.0,10,1,1,"#Vegan, #Repeat Borrower, #Supporting Family",male,true,bullet,field_partner
96,961161,Usman,English,Usman is a Babban Gona maize farmer. Thanks to...,Usman is a Babban Gona maize farmer. Thanks to...,200.0,200.0,funded,Personal Expenses,Personal Use,...,2015-11-12 00:39:38.000 +0000,14.0,8,2,2,#Vegan,male,true,bullet,field_partner
177,961701,Aminu,English,Aminu is a maize farmer and Babban Gona member...,Aminu is a maize farmer and Babban Gona member...,100.0,100.0,funded,Personal Expenses,Personal Use,...,2015-10-17 18:00:34.000 +0000,14.0,3,2,2,"#Parent, #Vegan",male,true,bullet,field_partner
207,1155776,Salisu,English,Salisu is a maize farmer from U/Dashire villag...,Salisu is a maize farmer from U/Dashire villag...,200.0,200.0,funded,Primary/secondary school costs,Education,...,2016-10-04 21:56:50.000 +0000,15.0,8,1,1,"#Schooling, #Vegan, #Repeat Borrower, #Parent",male,true,bullet,field_partner
217,1158968,Musa,English,Musa is a Babban Gona maize farmer from the vi...,Musa is a Babban Gona maize farmer from the vi...,125.0,125.0,funded,Personal Expenses,Personal Use,...,2016-10-03 20:56:13.000 +0000,15.0,5,1,1,"#Vegan, #Repeat Borrower",male,true,bullet,field_partner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419303,1067265,Juwyana,English,Juwayana is a Babban Gona maize farmer. She is...,Juwayana is a Babban Gona maize farmer. She is...,100.0,100.0,funded,Farming,Agriculture,...,2016-05-13 16:13:13.000 +0000,19.0,3,1,1,"#Parent, #Schooling, #Vegan",female,true,bullet,field_partner
1419304,1067985,Aliyu,English,Aliyu is a Babban Gona maize farmer from Turaw...,Aliyu is a Babban Gona maize farmer from Turaw...,350.0,350.0,funded,Farming,Agriculture,...,2016-06-11 13:46:05.000 +0000,19.0,14,1,1,"#Parent, #Schooling, #Vegan, #Sustainable Ag, ...",male,true,bullet,field_partner
1419374,1067288,Ubale,English,Ubale is a Babban Gona maize farmer. He is a h...,Ubale is a Babban Gona maize farmer. He is a h...,75.0,75.0,funded,Farming,Agriculture,...,2016-05-13 05:37:01.000 +0000,19.0,2,1,1,NaN,male,true,bullet,field_partner
1419443,1068602,Shehu,English,Shehu is a Babban Gona maize farmer from Kargi...,Shehu is a Babban Gona maize farmer from Kargi...,175.0,350.0,expired,Farming,Agriculture,...,NaN,19.0,6,1,1,"#Parent, #Schooling, #Vegan",male,true,bullet,field_partner


In [23]:
# decidiamo di invertire le date del "planned expiration time" con quelle del "disburse time" nel caso analizzato in precedenza
loans['planned_expiration_time'],loans['disburse_time']=np.where(loans['disburse_time'] > loans['planned_expiration_time'],(loans['disburse_time'],loans['planned_expiration_time']),(loans['planned_expiration_time'],loans['disburse_time']))

In [24]:
#verifichiamo che lo scambio è stato effettuato correttamente
loans.loc[loans['disburse_time'] > loans['planned_expiration_time']]

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model


In [25]:
# utilizziamo datatime per leggere le date come tali e non come stringhe
loans['planned_expiration_time'] =  pd.to_datetime(loans['planned_expiration_time'], format='%Y-%m-%d %H:%M:%S.%f')
loans['disburse_time'] =  pd.to_datetime(loans['disburse_time'], format='%Y-%m-%d %H:%M:%S.%f')

In [26]:
loans['planned_expiration_time'].head()

0   2014-02-14 03:30:06+00:00
1   2014-03-26 22:25:07+00:00
2   2014-02-15 21:10:05+00:00
3   2014-02-21 03:10:02+00:00
4   2014-02-13 06:10:02+00:00
Name: planned_expiration_time, dtype: datetime64[ns, UTC]

In [27]:
loans['disburse_time'].head()

0   2013-12-22 08:00:00+00:00
1   2013-12-20 08:00:00+00:00
2   2014-01-09 08:00:00+00:00
3   2014-01-17 08:00:00+00:00
4   2013-12-17 08:00:00+00:00
Name: disburse_time, dtype: datetime64[ns, UTC]

In [28]:
# calcoliamo la duration creando una nuova colonna
loans['duration']=loans['planned_expiration_time']-loans['disburse_time']

In [29]:
loans['duration'].head()

0   53 days 19:30:06
1   96 days 14:25:07
2   37 days 13:10:05
3   34 days 19:10:02
4   57 days 22:10:02
Name: duration, dtype: timedelta64[ns]

In [30]:
missingTime=loans[["planned_expiration_time","disburse_time", "duration"]]
missingTime.head()

,planned_expiration_time,disburse_time,duration
0,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,53 days 19:30:06
1,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,96 days 14:25:07
2,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,37 days 13:10:05
3,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,34 days 19:10:02
4,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,57 days 22:10:02


In [31]:
#verifichiamo che se una delle due date è mancante anche la duration è mancante
missingTime[missingTime['duration'].isna()]

,planned_expiration_time,disburse_time,duration
64,NaT,2010-11-18 08:00:00+00:00,NaT
65,NaT,2010-11-25 08:00:00+00:00,NaT
66,NaT,2010-11-25 08:00:00+00:00,NaT
67,NaT,2010-11-16 08:00:00+00:00,NaT
69,NaT,2010-11-10 08:00:00+00:00,NaT
...,...,...,...
1419523,NaT,2005-04-14 05:27:55+00:00,NaT
1419524,2017-02-09 23:07:54+00:00,NaT,NaT
1419525,2015-09-21 08:06:19+00:00,NaT,NaT
1419538,2017-02-25 20:21:21+00:00,NaT,NaT


# Punto 3: 

Find the lenders that have funded at least twice.


In [32]:
# raggruppiamo per lenders e contiamo quante volte è presente ciascuno di essi
twice=fin_fin.groupby(['lenders']).lenders.agg('count').to_frame('count').reset_index()

In [33]:
#selezioniamo i lenders presenti almeno due volte
twice.loc[twice['count'] >= 2]

,lenders,count
0,000,40
1,00000,39
2,0002,70
4,0101craign0101,71
5,0132575,4
...,...,...
1383792,zzinnia,28
1383793,zzivnk7094,4
1383794,zzmcfate,1730
1383796,zzrvmf8538,15


# Punto 4 

For each country, compute how many loans have involved that country as borrowers.


In [34]:
# raggruppiamo i loans per ogni Stato e vediamo quante volte è presente ciascuno di essi 
loans.groupby('country_code').size()

country_code
AF     2337
AL     3075
AM    13952
AZ    10172
BA      608
      ...  
XK     2178
YE     4206
ZA      633
ZM     1277
ZW     5513
Length: 95, dtype: int64

# Punto 5

For each country, compute the overall amount of money borrowed.


In [35]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,7.0,3,2,1,NaN,female,true,irregular,field_partner,53 days 19:30:06
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,8.0,11,2,1,NaN,female,true,monthly,field_partner,96 days 14:25:07
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days 13:10:05
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,34 days 19:10:02
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57 days 22:10:02


In [36]:
# dalla tabella loans abbiamo selezionato il country code e il loan amount
amount=loans[['country_code','loan_amount']]
amount.head()

,country_code,loan_amount
0,PH,125.0
1,HN,400.0
2,PK,400.0
3,KG,625.0
4,PH,425.0


In [37]:
#abbiamo raggruppato per country code e abbiamo sommato il loan amount
overall_cc=amount.groupby(['country_code']).sum()
overall_cc.head()

,loan_amount
country_code,
AF,1967950.0
AL,4307350.0
AM,22950475.0
AZ,14784625.0
BA,477250.0


# Punto 6

Like the previous point, but expressed as a percentage of the overall amount lent.


In [38]:
lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


In [39]:
# abbiamo creato una nuova colonna con le percentuali del loan per ogni stato sul totale
overall_cc['percentage']=(overall_cc['loan_amount']/overall_cc['loan_amount'].sum())*100
overall_cc['percentage']

country_code
AF    0.166578
AL    0.364598
AM    1.942654
AZ    1.251452
BA    0.040397
        ...   
XK    0.260964
YE    0.291519
ZA    0.085198
ZM    0.167511
ZW    0.495335
Name: percentage, Length: 95, dtype: float64

# Punto 7

Like the three previous points, but split for each year (with respect to disburse time).


In [40]:
anno = pd.DatetimeIndex(loans['disburse_time'])

In [41]:
loans['year']=anno.year

In [42]:
#abbiamo raggruppato per country code e abbiamo visualizzato il numero di loan per ogni anno
loans.groupby(['country_code','year']).size()

country_code  year  
AF            2007.0     408
              2008.0     370
              2009.0     678
              2010.0     632
              2011.0     247
                        ... 
ZW            2013.0     426
              2014.0    2078
              2015.0     600
              2016.0     808
              2017.0    1079
Length: 740, dtype: int64

In [43]:
amount_x_year=loans[['loan_amount',  'country_code', 'year']]

In [44]:
#abbiamo raggruppato per country code e year e abbiamo sommato loan amount
amount_x_year.groupby(['country_code','year']).sum()

loan_amount
country_code year               
AF           2007.0     194975.0
             2008.0     365375.0
             2009.0     585125.0
             2010.0     563350.0
             2011.0     245125.0
...                          ...
ZW           2013.0     678525.0
             2014.0    1311575.0
             2015.0     723625.0
             2016.0     788600.0
             2017.0    1237600.0

[740 rows x 1 columns]

In [45]:
#abbaimo calcolato la percentuale di loan diviso per country code nei diversi anni
overall_year=amount_x_year.groupby(['country_code','year']).sum()
overall_year['percentage']=(overall_year['loan_amount']/overall_year['loan_amount'].sum())*100
overall_year['percentage']

country_code  year  
AF            2007.0    0.016658
              2008.0    0.031216
              2009.0    0.049991
              2010.0    0.048131
              2011.0    0.020943
                          ...   
ZW            2013.0    0.057971
              2014.0    0.112057
              2015.0    0.061824
              2016.0    0.067375
              2017.0    0.105736
Name: percentage, Length: 740, dtype: float64

# Punto 8

For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.


In [46]:
# selezioniamo da loans solo le colonne loan id e loan amount
id_sum=loans[['loan_id',  'loan_amount']]
id_sum.head()

,loan_id,loan_amount
0,657307,125.0
1,657259,400.0
2,658010,400.0
3,659347,625.0
4,656933,425.0


In [47]:
fin_fin.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


In [48]:
# uniamo la tabella id_sum con la tabella normalizzata dei loan_lenders grazie all'attributo loan_id
each=pd.merge(fin_fin, id_sum, left_on="loan_id", right_on="loan_id")
each.head()

,loan_id,lenders,loan_amount
0,483693,muc888,1225.0
1,483693,sam4326,1225.0
2,483693,camaran3922,1225.0
3,483693,lachheb1865,1225.0
4,483693,rebecca3499,1225.0


In [49]:
# visualizziamo quante volte si ripete il loan_id per ottenere il numero di lender per ciascun loan
n_loans=each.groupby('loan_id').size()
n_loans

loan_id
84         3
85         2
86         3
88         3
89         4
          ..
1444051    1
1444053    1
1444058    1
1444063    1
1444065    1
Length: 1387428, dtype: int64

In [50]:
#creiamo il dataframe con il loan_id e il numero di lenders calcolato in precedenza
number = { 'number': n_loans } 
  
number2 = pd.DataFrame(number) 
number2

,number
loan_id,
84,3
85,2
86,3
88,3
89,4
...,...
1444051,1
1444053,1
1444058,1


In [51]:
# aggiungiamo alla tabella each il numero di lenders per ogni loan
each_number=pd.merge(each, number2, left_on="loan_id", right_on="loan_id")
each_number.head()

,loan_id,lenders,loan_amount,number
0,483693,muc888,1225.0,40
1,483693,sam4326,1225.0,40
2,483693,camaran3922,1225.0,40
3,483693,lachheb1865,1225.0,40
4,483693,rebecca3499,1225.0,40


In [52]:
# Dividendo il loan amount con il numero di lenders per loan otteniamo le quote individuali per ciascun lenders
each_number['individual']=each_number['loan_amount']/each_number['number']
each_number.head()

,loan_id,lenders,loan_amount,number,individual
0,483693,muc888,1225.0,40,30.625
1,483693,sam4326,1225.0,40,30.625
2,483693,camaran3922,1225.0,40,30.625
3,483693,lachheb1865,1225.0,40,30.625
4,483693,rebecca3499,1225.0,40,30.625


In [53]:
#selezioniamo dalla tabella precedente gli attributi lenders e individual
sin_lender=each_number[['lenders', 'individual']]

In [54]:
#raggruppiamo i lender per ottenere il totale dei finanziamenti effettuati da ognuno
sum_lenders=sin_lender.groupby(['lenders']).sum().reset_index(drop=False)
sum_lenders

,lenders,individual
0,000,1764.285078
1,00000,1380.693644
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117
...,...,...
1383794,zzmcfate,66113.226325
1383795,zzpaghetti9994,51.020408
1383796,zzrvmf8538,576.978086
1383797,zzzsai,267.667370


# Punto 9

For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.


In [55]:
lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


In [56]:
# selezioniamo gli attributi permanent name e country code
lend=lenders[['permanent_name','country_code']].copy()
lend.head()

,permanent_name,country_code
0,qian3013,NaN
1,reena6733,NaN
2,mai5982,NaN
3,andrew86079135,NaN
4,nguyen6962,NaN


In [57]:
# notiamo che nella tabella lenders più di un milione di righe non contengono il country code
# abbiamo deciso di calcolare la distribuzione dei country code all'interno della tabella lenders e abbiamo riempito i dati mancanti seguendo la stessa distribuzione
lend[lend['country_code'].isna()]

,permanent_name,country_code
0,qian3013,NaN
1,reena6733,NaN
2,mai5982,NaN
3,andrew86079135,NaN
4,nguyen6962,NaN
...,...,...
2349167,todd5695,NaN
2349168,kate40761039,NaN
2349169,janet7309,NaN
2349170,pj4198,NaN


*Inizio del calcolo per l'imputazione*

In [58]:
# raggruppiamo per country code per sapere quante volte è presente ciascuno stato
frequency=lend.groupby(['country_code']).size()
frequency

country_code
AD      15
AE    1043
AF     228
AG       8
AI       4
      ... 
YE     195
YT       2
ZA    1051
ZM      65
ZW      54
Length: 234, dtype: int64

In [59]:
#abbiamo creato una tabella utilizzando i dati ottenuti in precedenza
c_f={"frequency":frequency}
country_freq=pd.DataFrame(c_f)

country_freq['country_code']=country_freq.index
country_freq=country_freq.reset_index(drop=True)

country_freq=country_freq[['country_code','frequency']]
country_freq.head()

,country_code,frequency
0,AD,15
1,AE,1043
2,AF,228
3,AG,8
4,AI,4


In [60]:
country_freq['percentage']=(country_freq['frequency']/country_freq['frequency'].sum())*100
country_freq.head()

,country_code,frequency,percentage
0,AD,15,0.001684
1,AE,1043,0.117120
2,AF,228,0.025602
3,AG,8,0.000898
4,AI,4,0.000449


In [61]:
#creiamo due array con i volori della percentuale e del country code per poterli utilizzare nella funzione successiva
percentage=country_freq['percentage'].to_numpy()
country_code=country_freq['country_code'].to_numpy()
percentage /= percentage.sum()

In [62]:
#riempiamo i dati mancanti come spiegato precedentemente
lend['country_code'] = lend['country_code'].fillna(pd.Series(np.random.choice(
                                                     country_code,p=country_freq.percentage, size=len(lend))))

*fine dell'imputazione*

In [63]:
lend.head()

,permanent_name,country_code
0,qian3013,US
1,reena6733,US
2,mai5982,CA
3,andrew86079135,ES
4,nguyen6962,AU


In [64]:
sum_lenders.head()

,lenders,individual
0,000,1764.285078
1,00000,1380.693644
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117


In [65]:
# uniamo le tabella sum lenders e lend
# ipotizziamo che il permanent name della tabella lenders corrisponda al loan id della tabella loans
inner_sum_lenders=pd.merge(sum_lenders, lend, left_on="lenders", right_on="permanent_name", how='inner')
inner_sum_lenders.head()

,lenders,individual,permanent_name,country_code
0,000,1764.285078,000,US
1,00000,1380.693644,00000,DE
2,0002,2472.563566,0002,US
3,00mike00,52.631579,00mike00,US
4,0101craign0101,2623.565117,0101craign0101,US


In [66]:
# Raggruppando per country code e sommando abbiamo trovato il totale della somma prestata per ciascuno stato
money_lent=inner_sum_lenders.groupby(['country_code']).sum().reset_index(drop=False)
money_lent.rename(columns={'individual':'money_lent'}, inplace=True)
money_lent.head()

,country_code,money_lent
0,AD,9.396155e+03
1,AE,2.315392e+06
2,AF,1.713722e+05
3,AG,3.213767e+03
4,AI,8.089221e+02


In [67]:
#ipotizziamo che il country code nella tabella loans faccia riferimenti al country code di chi riceve il prestito
# Raggruppando per country code e sommando abbiamo trovato il totale della somma presa in prestito da ciascuno stato
money_borrowed=overall_cc.groupby(['country_code']).sum().reset_index(drop=False)
money_borrowed.rename(columns={'loan_amount':'money_borrowed'}, inplace=True)
money_borrowed.head()

,country_code,money_borrowed,percentage
0,AF,1967950.0,0.166578
1,AL,4307350.0,0.364598
2,AM,22950475.0,1.942654
3,AZ,14784625.0,1.251452
4,BA,477250.0,0.040397


In [68]:
#abbiamo unito le due tabelle per poter fare la differenza tra le due somme
money_both = money_lent.merge(money_borrowed,how='left', on='country_code')
money_both.head()

,country_code,money_lent,money_borrowed,percentage
0,AD,9.396155e+03,NaN,NaN
1,AE,2.315392e+06,NaN,NaN
2,AF,1.713722e+05,1967950.0,0.166578
3,AG,3.213767e+03,NaN,NaN
4,AI,8.089221e+02,NaN,NaN


In [69]:
money_both.money_borrowed = money_both.money_borrowed.fillna(0)
money_both.head()

,country_code,money_lent,money_borrowed,percentage
0,AD,9.396155e+03,0.0,NaN
1,AE,2.315392e+06,0.0,NaN
2,AF,1.713722e+05,1967950.0,0.166578
3,AG,3.213767e+03,0.0,NaN
4,AI,8.089221e+02,0.0,NaN


In [70]:
money_both['delta'] = money_both['money_lent'] - money_both['money_borrowed']

In [71]:
money_both.head()

,country_code,money_lent,money_borrowed,percentage,delta
0,AD,9.396155e+03,0.0,NaN,9.396155e+03
1,AE,2.315392e+06,0.0,NaN,2.315392e+06
2,AF,1.713722e+05,1967950.0,0.166578,-1.796578e+06
3,AG,3.213767e+03,0.0,NaN,3.213767e+03
4,AI,8.089221e+02,0.0,NaN,8.089221e+02


# Punto 10

Which country has the highest ratio between the difference computed at the previous point and the population?


In [72]:
#selezioniamo gli attributi country code e population
c_stat2=c_stat[['country_code', 'population']]
c_stat2.head()

,country_code,population
0,IN,1339180127
1,NG,190886311
2,MX,129163276
3,PK,197015955
4,BD,164669751


In [73]:
# uniamo la tabella contenetente i soldi prestati e presi in prestito con le statistiche per paese
final_stat=pd.merge(money_both, c_stat2, left_on="country_code", right_on="country_code")
final_stat.head()

,country_code,money_lent,money_borrowed,percentage,delta,population
0,AE,2.315392e+06,0.0,NaN,2.315392e+06,9400145
1,AF,1.713722e+05,1967950.0,0.166578,-1.796578e+06,35530081
2,AL,3.206182e+04,4307350.0,0.364598,-4.275288e+06,2930187
3,AM,4.481651e+04,22950475.0,1.942654,-2.290566e+07,2930450
4,AO,4.337952e+04,0.0,NaN,4.337952e+04,29784193


In [74]:
#calcoliamo per ogni paese il rapporto tra la differenza precedentemente calcolata e la population
final_stat['ratio']=final_stat['delta']/final_stat['population']
final_stat.head()

,country_code,money_lent,money_borrowed,percentage,delta,population,ratio
0,AE,2.315392e+06,0.0,NaN,2.315392e+06,9400145,0.246314
1,AF,1.713722e+05,1967950.0,0.166578,-1.796578e+06,35530081,-0.050565
2,AL,3.206182e+04,4307350.0,0.364598,-4.275288e+06,2930187,-1.459050
3,AM,4.481651e+04,22950475.0,1.942654,-2.290566e+07,2930450,-7.816430
4,AO,4.337952e+04,0.0,NaN,4.337952e+04,29784193,0.001456


In [75]:
#selezioniamo il paese con il valore del rapporto più alto
final_stat[final_stat['ratio']==final_stat['ratio'].max()]

,country_code,money_lent,money_borrowed,percentage,delta,population,ratio
114,NO,2.066148e+07,0.0,NaN,2.066148e+07,5305383,3.894437


# Punto 11

Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?


In [76]:
final_stat.head()

,country_code,money_lent,money_borrowed,percentage,delta,population,ratio
0,AE,2.315392e+06,0.0,NaN,2.315392e+06,9400145,0.246314
1,AF,1.713722e+05,1967950.0,0.166578,-1.796578e+06,35530081,-0.050565
2,AL,3.206182e+04,4307350.0,0.364598,-4.275288e+06,2930187,-1.459050
3,AM,4.481651e+04,22950475.0,1.942654,-2.290566e+07,2930450,-7.816430
4,AO,4.337952e+04,0.0,NaN,4.337952e+04,29784193,0.001456


In [77]:
pop_below=c_stat[['country_code','population_below_poverty_line']]
pop_below.head()

,country_code,population_below_poverty_line
0,IN,21.9
1,NG,70.0
2,MX,46.2
3,PK,29.5
4,BD,31.5


In [78]:
#aggiungiamo la tabella del punto 11 la percentuale della popolazione sotto la soglia della povertà
pop_merged=pd.merge(final_stat, pop_below, left_on="country_code", right_on="country_code")
del pop_merged['ratio']
pop_merged.head()

,country_code,money_lent,money_borrowed,percentage,delta,population,population_below_poverty_line
0,AE,2.315392e+06,0.0,NaN,2.315392e+06,9400145,19.5
1,AF,1.713722e+05,1967950.0,0.166578,-1.796578e+06,35530081,35.8
2,AL,3.206182e+04,4307350.0,0.364598,-4.275288e+06,2930187,14.3
3,AM,4.481651e+04,22950475.0,1.942654,-2.290566e+07,2930450,32.0
4,AO,4.337952e+04,0.0,NaN,4.337952e+04,29784193,40.5


In [79]:
#calcoliamo per ogni paese il numero di persone sopra la soglia della povertà
pop_merged['pop_upper']=pop_merged['population']*(1-(pop_merged['population_below_poverty_line']/100))
pop_merged.head()

,country_code,money_lent,money_borrowed,percentage,delta,population,population_below_poverty_line,pop_upper
0,AE,2.315392e+06,0.0,NaN,2.315392e+06,9400145,19.5,7.567117e+06
1,AF,1.713722e+05,1967950.0,0.166578,-1.796578e+06,35530081,35.8,2.281031e+07
2,AL,3.206182e+04,4307350.0,0.364598,-4.275288e+06,2930187,14.3,2.511170e+06
3,AM,4.481651e+04,22950475.0,1.942654,-2.290566e+07,2930450,32.0,1.992706e+06
4,AO,4.337952e+04,0.0,NaN,4.337952e+04,29784193,40.5,1.772159e+07


In [80]:
#calcoliamo per ogni country code il rapporto tra la differenza tra i soldi prestati e presi in prestito e la popolazione sopra la soglia della povertà 
pop_merged['up_ratio']=pop_merged['delta']/pop_merged['pop_upper']
pop_merged.head()

,country_code,money_lent,money_borrowed,percentage,delta,population,population_below_poverty_line,pop_upper,up_ratio
0,AE,2.315392e+06,0.0,NaN,2.315392e+06,9400145,19.5,7.567117e+06,0.305981
1,AF,1.713722e+05,1967950.0,0.166578,-1.796578e+06,35530081,35.8,2.281031e+07,-0.078762
2,AL,3.206182e+04,4307350.0,0.364598,-4.275288e+06,2930187,14.3,2.511170e+06,-1.702508
3,AM,4.481651e+04,22950475.0,1.942654,-2.290566e+07,2930450,32.0,1.992706e+06,-11.494751
4,AO,4.337952e+04,0.0,NaN,4.337952e+04,29784193,40.5,1.772159e+07,0.002448


In [81]:
#selezioniamo il country code con il rapporto più elevato
pop_merged[pop_merged['up_ratio']==pop_merged['up_ratio'].max()]

,country_code,money_lent,money_borrowed,percentage,delta,population,population_below_poverty_line,pop_upper,up_ratio
24,CA,9.382434e+07,50000.0,0.004232,9.377434e+07,36624199,9.4,3.318152e+07,2.826101


# Punto 12


For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.


In [82]:
#eliminiamo dalla tabella loans le righe che presentano dati mancanti per gli attributi: 'loan_id', 'loan_amount', 'planned_expiration_time', 'disburse_time'
loans_noNA=loans[['loan_id', 'loan_amount', 'planned_expiration_time', 'disburse_time']].dropna()

In [83]:
loans_noNA.head()

,loan_id,loan_amount,planned_expiration_time,disburse_time
0,657307,125.0,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00
1,657259,400.0,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00
2,658010,400.0,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00
3,659347,625.0,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00
4,656933,425.0,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00


In [84]:
loans12=loans_noNA

In [85]:
#aggiungiamo a loans una colonna che contiene solo l'anno dell'attributo disburse time
loans12['disb_year'] = loans12.disburse_time.apply(lambda x: x.year)
loans12['disb_year'].head()

0    2013
1    2013
2    2014
3    2014
4    2013
Name: disb_year, dtype: int64

In [86]:
#aggiungiamo a loans una colonna che contiene solo l'anno dell'attributo expiration time
loans12['exp_year'] = loans12.planned_expiration_time.apply(lambda x: x.year)
loans12['exp_year'].head()

0    2014
1    2014
2    2014
3    2014
4    2014
Name: exp_year, dtype: int64

In [87]:
#calcoliamo la differenza tra l'anno dell'expiration e del disburse time 
loans12['diff_in_years'] = loans12['exp_year'] - loans12['disb_year']
loans12['diff_in_years'].head()

0    1
1    1
2    0
3    0
4    1
Name: diff_in_years, dtype: int64

In [88]:
# troviamo a data al 12/31 dell'anno presente in disburse time
loans12['end'] = loans12.disburse_time.apply(lambda d: d + pd.offsets.YearEnd())
loans12['end'].head()

0   2013-12-31 08:00:00+00:00
1   2013-12-31 08:00:00+00:00
2   2014-12-31 08:00:00+00:00
3   2014-12-31 08:00:00+00:00
4   2013-12-31 08:00:00+00:00
Name: end, dtype: datetime64[ns, UTC]

In [89]:
# troviamo a data al 01/01 dell'anno presente in expiration time
loans12['begin'] = loans12.planned_expiration_time.apply(lambda d: d - pd.offsets.YearBegin())
loans12['begin'].head()

0   2014-01-01 03:30:06+00:00
1   2014-01-01 22:25:07+00:00
2   2014-01-01 21:10:05+00:00
3   2014-01-01 03:10:02+00:00
4   2014-01-01 06:10:02+00:00
Name: begin, dtype: datetime64[ns, UTC]

In [90]:
#calcoliamo i giorni tra la fine di ogni anno e il disburse time
loans12['day_from_end'] = (loans12.end - loans12.disburse_time).dt.days
loans12['day_from_end'].head()

0      9
1     11
2    356
3    348
4     14
Name: day_from_end, dtype: int64

In [91]:
#calcoliamo i giorni tra l'inizio di ogni anno e l'expiration time
loans12['day_from_begin'] = (loans12.planned_expiration_time - loans12.begin).dt.days
loans12['day_from_begin'].head()

0    44
1    84
2    45
3    51
4    43
Name: day_from_begin, dtype: int64

In [92]:
loans12.head()

,loan_id,loan_amount,planned_expiration_time,disburse_time,disb_year,exp_year,diff_in_years,end,begin,day_from_end,day_from_begin
0,657307,125.0,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 03:30:06+00:00,9,44
1,657259,400.0,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 22:25:07+00:00,11,84
2,658010,400.0,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,2014,2014,0,2014-12-31 08:00:00+00:00,2014-01-01 21:10:05+00:00,356,45
3,659347,625.0,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,2014,2014,0,2014-12-31 08:00:00+00:00,2014-01-01 03:10:02+00:00,348,51
4,656933,425.0,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,2013,2014,1,2013-12-31 08:00:00+00:00,2014-01-01 06:10:02+00:00,14,43


In [93]:
#creiamo una lista vuota
list_of_dict = []

In [94]:
def function12(data):
    
    day_from_end=data.day_from_end
    day_from_begin=data.day_from_begin
    
    #trasformiamo la differenza di anni in giorni 
    #sottraiamo 365 per non contare l'anno dell'expiration
    year_between = (365*data.diff_in_years) - 365
    
    #calcoliamo il totale dei giorni tra disburse e expiration
    total_day = day_from_begin+day_from_end+year_between
    
    day_in_year=365

    disb_year = data.disb_year
    exp_year = data.exp_year
    
    # global serve per richiamare la lista fuori della funzione
    global list_of_dict

    # se la differenza in anni tra disburse e expiration è 0 entra nell'if 
    if data.diff_in_years == 0:
        
        loan_x_year = data.loan_amount
        years = data.disb_year
        combo = {'amount': loan_x_year,'years':years}
        list_of_dict.append(combo)
        
    #se la differenza è diversa da 0 entra nell'else
    else:
        
        #con range prendiamo in considerazione gli anni tra disburse ed expiration (escluso)
        for x in range(disb_year,exp_year,1):
            loan_x_year = (data.loan_amount * day_from_end)/ total_day
            combo = {'amount': loan_x_year,'years':disb_year}
            list_of_dict.append(combo)
            
            # se la differenza di anni è maggiore di uno, per gli anni che non sono ne quelli del disburse ne quelli 
            #dell'expiration il numero di giorni è 365  
            day_from_end=day_in_year
            disb_year=disb_year+1
        
        #calcola la quota dei giorni dell'expiration             
        loan_x_year = (data.loan_amount * day_from_begin)/ total_day
        years = data.exp_year
        combo = {'amount': loan_x_year,'years':years}
        list_of_dict.append(combo)
     
    return 

In [95]:
# applico la funzione a loans12
loans12.apply(lambda x : function12(x), axis='columns')

0          None
1          None
2          None
3          None
4          None
           ... 
1419602    None
1419603    None
1419604    None
1419605    None
1419606    None
Length: 1044962, dtype: object

In [96]:
list_of_dict

[{'amount': 21.22641509433962, 'years': 2013},
 {'amount': 103.77358490566037, 'years': 2014},
 {'amount': 46.31578947368421, 'years': 2013},
 {'amount': 353.6842105263158, 'years': 2014},
 {'amount': 400.0, 'years': 2014},
 {'amount': 625.0, 'years': 2014},
 {'amount': 104.3859649122807, 'years': 2013},
 {'amount': 320.6140350877193, 'years': 2014},
 {'amount': 350.0, 'years': 2014},
 {'amount': 125.0, 'years': 2014},
 {'amount': 1600.0, 'years': 2014},
 {'amount': 350.0, 'years': 2014},
 {'amount': 1825.0, 'years': 2014},
 {'amount': 200.0, 'years': 2014},
 {'amount': 950.0, 'years': 2014},
 {'amount': 159.32203389830508, 'years': 2013},
 {'amount': 1015.6779661016949, 'years': 2014},
 {'amount': 16.853932584269664, 'years': 2013},
 {'amount': 283.14606741573033, 'years': 2014},
 {'amount': 400.0, 'years': 2014},
 {'amount': 225.0, 'years': 2014},
 {'amount': 1500.0, 'years': 2014},
 {'amount': 125.0, 'years': 2014},
 {'amount': 375.0, 'years': 2015},
 {'amount': 225.0, 'years': 2015

In [97]:
df12 = pd.DataFrame(list_of_dict) 
df12.head()

,amount,years
0,21.226415,2013
1,103.773585,2014
2,46.315789,2013
3,353.684211,2014
4,400.000000,2014


In [98]:
df12.groupby('years').sum()

,amount
years,
2011,6.646341e+05
2012,1.133961e+08
2013,1.244435e+08
2014,1.537562e+08
2015,1.597879e+08
2016,1.592698e+08
2017,1.725565e+08
2018,5.929418e+06


# Punto 13


For each value of repayment_interval, add a new column to the lenders dataframe that contains the total amount of money corresponding to loans in such state


In [99]:
#selezioniamo gli attributi loan id e repayment interval dalla tabella loans
rep=loans[['loan_id','repayment_interval']]
rep.head()

,loan_id,repayment_interval
0,657307,irregular
1,657259,monthly
2,658010,monthly
3,659347,monthly
4,656933,bullet


In [100]:
# selezioniamo dalla tabella del punto 8 gli attributi: 'loan_id','lenders', 'individual'
e_n=each_number[['loan_id','lenders', 'individual']]
e_n.head()

,loan_id,lenders,individual
0,483693,muc888,30.625
1,483693,sam4326,30.625
2,483693,camaran3922,30.625
3,483693,lachheb1865,30.625
4,483693,rebecca3499,30.625


In [102]:
#Uniamo le due tabelle
repayment=pd.merge(e_n, rep, left_on="loan_id", right_on="loan_id").drop("loan_id", axis=1)
repayment.head()

,lenders,individual,repayment_interval
0,muc888,30.625,irregular
1,sam4326,30.625,irregular
2,camaran3922,30.625,irregular
3,lachheb1865,30.625,irregular
4,rebecca3499,30.625,irregular


In [103]:
# unstack invertiva aggiunge i valori di repayment interval come attributi alla tabella precedente
repayment2=repayment.groupby(['lenders','repayment_interval'])['individual'].sum().unstack()

In [104]:
repayment2=repayment2.fillna(0)
repayment2=repayment2.reset_index()
repayment2.head()

repayment_interval,lenders,bullet,irregular,monthly,weekly
0,000,143.343126,306.649935,1314.292018,0.0
1,00000,106.506211,551.184378,723.003055,0.0
2,0002,209.410885,1165.269449,1097.883232,0.0
3,00mike00,0.000000,0.000000,52.631579,0.0
4,0101craign0101,126.162029,531.016520,1966.386567,0.0


In [105]:
# uniamo la tabella precedente con lenders
final13=pd.merge(lenders, repayment2, left_on="permanent_name", right_on="lenders").drop("lenders", axis=1)
final13

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited,bullet,irregular,monthly,weekly
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0,0.0,46.839080,0.000000,0.0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0,0.0,32.471264,385.477101,0.0
2,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0,0.0,0.000000,349.601088,0.0
3,sirinapa6764,Sirinapa,NaN,NaN,NaN,1461301496,NaN,NaN,1.0,NaN,0,0.0,0.000000,50.000000,0.0
4,rene7585,Rene,NaN,NaN,NaN,1461301636,NaN,NaN,2.0,NaN,0,0.0,0.000000,59.126984,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383794,carol8279,Carol,NaN,NaN,NaN,1342099416,NaN,NaN,5.0,NaN,0,0.0,67.164905,102.275483,0.0
1383795,eric91401545,Eric,NaN,NaN,NaN,1342100719,NaN,NaN,2.0,NaN,0,0.0,36.280488,27.500000,0.0
1383796,kate40761039,Kate,NaN,NaN,NaN,1342096938,NaN,NaN,1.0,Irina,0,0.0,36.280488,0.000000,0.0
1383797,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0,0.0,83.333333,27.173913,0.0


# Punto 14

What is the occupation with the highest average amount of money lent (the average must be computed over all lenders with a given occupation)?


In [106]:
sum_lenders.head()

,lenders,individual
0,000,1764.285078
1,00000,1380.693644
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117


In [107]:
lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


In [108]:
#selezioniamo dalla tabella lenders solo gli attributi: 'permanent_name','occupation'
lenders14=lenders[['permanent_name','occupation']]

In [109]:
#aggiungiamo alla tabella precedente le quote di ogni lender
occ=pd.merge(lenders14, sum_lenders, left_on="permanent_name", right_on="lenders")
occ

,permanent_name,occupation,lenders,individual
0,qian3013,NaN,qian3013,46.839080
1,reena6733,NaN,reena6733,417.948366
2,andrew86079135,NaN,andrew86079135,349.601088
3,sirinapa6764,NaN,sirinapa6764,50.000000
4,rene7585,NaN,rene7585,59.126984
...,...,...,...,...
1383794,carol8279,NaN,carol8279,169.440389
1383795,eric91401545,NaN,eric91401545,63.780488
1383796,kate40761039,NaN,kate40761039,36.280488
1383797,maria2141,NaN,maria2141,110.507246


In [110]:
#elimina spazi e caratteri speciali all'inizio della stringa
occ.occupation= occ.occupation.str.replace(r'^[\s*\W*]+', r'', regex=True)

In [111]:
#elimina spazi e caratteri speciali alla fine della stringa
occ.occupation= occ.occupation.str.replace(r'[\s*\W*]+$', r'', regex=True)

In [112]:
#se è presente più di uno spazio bianco tra una parola e l'altra ne lascia solo uno
occ.occupation= occ.occupation.str.replace(r'[\s*]', r' ', regex=True)

In [113]:
#trasforma stringhe composte unicamente da spazi bianchi in NaN
occ_WS=occ.replace(r'^\s*$', np.nan, regex=True)

In [114]:
#eliminiamo le righe con occupation mancante
occ_noNA=occ_WS.dropna(subset=['occupation'])
occ_noNA.head()

,permanent_name,occupation,lenders,individual
31,vikas1098,Software Engineer,vikas1098,91.666667
177,kumari2781,Software Engineer,kumari2781,63.873626
390,javier7867,Technology Consultant,javier7867,457.002956
945,jens1183,IT Consultant,jens1183,59.343605
1099,pankaj1930,doctor,pankaj1930,137.546380


In [115]:
# raggruppiamo per occupation e abbiamo sostituito le maiuscole con le minuscole e abbiamo fatto la media delle quote individuali tra le stesse occupation
avg_occ=occ_noNA.groupby(occ_noNA.occupation.str.lower()).mean().reset_index()
avg_occ

,occupation,individual
0,0.167mb,1870.116664
1,1 pika,34.302326
2,1 st grader,47.619048
3,1 year old,198.329293
4,1) hotel management - 2) b ict,89.515129
...,...,...
68715,技術者,153885.830846
68716,接摩師,511.049861
68717,電腦工程師,79.807692
68718,音楽家,25.000000


In [116]:
avg_occ.loc[avg_occ['individual'].idxmax()]

occupation    www.linkedin.com/in/peacekeeper
individual                         8.6425e+06
Name: 68333, dtype: object

In [117]:
#abbiamo selezionato l'occupazione con la media maggiore
avg_occ[avg_occ['individual']==avg_occ['individual'].max()]

,occupation,individual
68333,www.linkedin.com/in/peacekeeper,8.642502e+06


# Punto 15

Cluster the loans according to the year-month of disburse time.


In [118]:
loans4time=loans

In [119]:
#eliminiamo le righe con disburse_time mancante
loans4time=loans4time.dropna(subset=['disburse_time']).copy()

In [120]:
#abbiamo selezionato l'anno del disburse time
loans4time['year']=loans4time['disburse_time'].dt.year

In [121]:
#abbiamo selezionato il mese del disburse time
loans4time['month']=loans4time['disburse_time'].dt.month

In [122]:
year_month=loans4time.set_index(['year','month','loan_id'])
year_month.sort_index(inplace=True)
year_month

loan_name original_language  \
year month loan_id                                    
2005 4     84             Justine           English   
           85            Geoffrey           English   
           86              Apollo           English   
           88              Eunice           English   
           89                Rose           English   
...                           ...               ...   
2018 1     1444084            NaN           English   
           1444085            NaN           English   
     2     1443192  BELSY IMALVER           Spanish   
           1443194         EDILMA           Spanish   
           1443196        NORELCY           Spanish   

                                                          description  \
year month loan_id                                                      
2005 4     84       <i>The following description was written by Mo...   
           85       <i>The following description was written by Mo...   
           86       <i>The following description was written by Mo...   
           88       Eunice received 100 dollar grant from Village ...   
           89       <i>\r\nThe following description was written b...   
...                                                               ...   
2018 1     1444084  To pay for feed and a hutch for the turduckens...   
           1444085  To pay for feed and a hutch for the turduckens...   
     2     1443192  Belcy es una madre cabeza de hogar de 35 años ...   
           1443194  Edilma es una madre cabeza de hogar de 62 años...   
           1443196  Norelcy tiene 40 años de edad, es madre cabeza...   

                                               description_translated  \
year month loan_id                                                      
2005 4     84                                                     NaN   
           85                                                     NaN   
           86                                                     NaN   
           88                                                     NaN   
           89                                                     NaN   
...                                                               ...   
2018 1     1444084  To pay for feed and a hutch for the turduckens...   
           1444085  To pay for feed and a hutch for the turduckens...   
     2     1443192  Cocktail lounge, Norway: Ladies are requested ...   
           1443194  Cocktail lounge, Norway: Ladies are requested ...   
           1443196  Cocktail lounge, Norway: Ladies are requested ...   

                    funded_amount  loan_amount       status  \
year month loan_id                                            
2005 4     84               500.0        500.0       funded   
           85               500.0        500.0       funded   
           86               500.0        500.0       funded   
           88               300.0        300.0       funded   
           89               500.0        500.0       funded   
...                           ...          ...          ...   
2018 1     1444084            0.0         25.0  fundRaising   
           1444085            0.0         25.0  fundRaising   
     2     1443192            0.0        525.0  fundRaising   
           1443194            0.0        525.0  fundRaising   
           1443196            0.0        525.0  fundRaising   

                            activity_name  sector_name  \
year month loan_id                                       
2005 4     84                Butcher Shop         Food   
           85       Food Production/Sales         Food   
           86                Animal Sales  Agriculture   
           88              Clothing Sales     Clothing   
           89       Food Production/Sales         Food   
...                                   ...          ...   
2018 1     1444084              Livestock  Agriculture   
           1444085              Livestock  Agriculture   
     2  

# Punto 16

For each country, compute its overall GNI, by multiplying the per capita GNI with its population.


In [123]:
gni_stat=c_stat
gni_stat

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Somalia,SO,SOM,Africa,Eastern Africa,14742523,NaN,NaN,NaN,NaN,NaN,NaN,Somalia
170,Central African Republic,CF,CAF,Africa,Middle Africa,4659080,NaN,0.352440,51.458,7.098980,4.230000,587.473961,Central African Republic
171,Samoa,WS,WSM,Oceania,Polynesia,196440,NaN,0.702000,NaN,NaN,NaN,NaN,Samoa
172,Palestine,PS,PS,Asia,Western Asia,4920724,NaN,0.677000,NaN,NaN,NaN,NaN,Palestine


In [124]:
# verifichiamo che alcuni stati presentano gni mancante
gni_stat[gni_stat['gni'].isna()]

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
161,North Korea,KP,PRK,Asia,Eastern Asia,25490965,NaN,NaN,NaN,NaN,NaN,NaN,North Korea
164,Puerto Rico,PR,PRI,Americas,Caribbean,3663131,NaN,NaN,NaN,NaN,NaN,NaN,Puerto Rico
169,Somalia,SO,SOM,Africa,Eastern Africa,14742523,NaN,NaN,NaN,NaN,NaN,NaN,Somalia
171,Samoa,WS,WSM,Oceania,Polynesia,196440,NaN,0.702,NaN,NaN,NaN,NaN,Samoa
172,Palestine,PS,PS,Asia,Western Asia,4920724,NaN,0.677,NaN,NaN,NaN,NaN,Palestine
173,Kosovo,XK,NaN,Europe,Southern Europe,1895250,30.0,0.786,NaN,NaN,NaN,NaN,Kosovo


In [125]:
#decidiamo di cercare i valori mancanti utilizzando la fonte indicata su Kaggle 

    # KIVA: http://hdr.undp.org/en/composite/HDI [per Palestina e Samoa]
# abbiamo trovato i valori dello gni del Porto Rico e Kosovo in quanto mancanti anche nella fonte citata sopra
    # WORLD BANK: https://data.worldbank.org/indicator/NY.GDP.PCAP.CD?end=2017&locations=KP&start=1960 [per Porto Rico e Kosovo]
    
gni_stat.at[164,'gni']= 26190 #Puerto Rico
gni_stat.at[171,'gni']= 4258.5 #Samoa
gni_stat.at[172,'gni']= 5055 #Palestine
gni_stat.at[173,'gni']= 10970 #Kosovo


In [126]:
# i dati della Korea del Nord e della Somalia nel 2017 non sono disponibili da nessuna fonte
gni_stat=gni_stat.dropna(subset=['gni'])

In [127]:
#seleziono gli attributi: 'country_name','population','gni'
gni_stat2=gni_stat[['country_name','population','gni']].copy()

In [128]:
#calcolo l'overall gni
gni_stat2['overall_gni'] = gni_stat2['population'] * gni_stat2['gni']
gni_stat2.head()

,country_name,population,gni,overall_gni
0,India,1339180127,5663.474799,7.584413e+12
1,Nigeria,190886311,5442.901264,1.038975e+12
2,Mexico,129163276,16383.106680,2.116096e+12
3,Pakistan,197015955,5031.173074,9.912214e+11
4,Bangladesh,164669751,3341.490722,5.502424e+11
